In [6]:
# Adaptation of https://elitedatascience.com/python-machine-learning-tutorial-scikit-learn
import dds
import sklearn
import pandas as pd
import logging
logging.basicConfig(level=logging.DEBUG)

import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import requests
import io
import json


In [7]:
# debug
import importlib
import inspect
"dds" in importlib.import_module("__main__").__dict__

True

In [8]:
path_raw = "/wine-quality/raw"
path_model = "/wine-quality/my_model"
path_model_stats = "/wine-quality/my_model_stats.json"

def _load_data():
    url = "https://raw.githubusercontent.com/zygmuntz/wine-quality/master/winequality/winequality-red.csv"
    x = requests.get(url=url, verify=False).content 
    return pd.read_csv(io.StringIO(x.decode('utf8')), sep=";")

def load_data():
    return dds.keep(path_raw, _load_data)

data = dds.eval(load_data)

data.head(3)
    

DEBUG:dds:locals: ['GridSearchCV', 'In', 'Out', 'RandomForestRegressor', '_', '_2', '_3', '_7', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_ih', '_ii', '_iii', '_load_data', '_oh', 'build_model', 'data', 'dds', 'exit', 'get_ipython', 'importlib', 'inspect', 'io', 'json', 'load_data', 'logging', 'make_pipeline', 'mean_squared_error', 'model_stats', 'np', 'path_model', 'path_model_stats', 'path_raw', 'pd', 'pipeline', 'preprocessing', 'quit', 'r2_score', 'requests', 'sklearn', 'train_test_split']
DEBUG:dds.introspect:Starting _introspect: <function load_data at 0x7f21cead4a60>: arg_sig=()
DEBUG:dds.introspect:visit name dds: skipping (fun/mod)
DEBUG:dds.introspect:is_authorized_path: {'__main__', 'dds'}
DEBUG:dds.introspect:visit name path_raw: authorized
DEBUG:dds.introspect:visit name _load_data: skipping (fun/mod)
DEBUG:dds.introspect:local vars: []
DE

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5


In [9]:

def build_model(X_train, y_train):
    pipeline = make_pipeline(preprocessing.StandardScaler(), 
                             RandomForestRegressor(n_estimators=100))
    hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                      'randomforestregressor__max_depth': [None, 5, 3, 1]}

    clf = GridSearchCV(pipeline, hyperparameters, cv=10)
    clf.fit(X_train, y_train)
    return clf
 
    
def model_stats(clf, X_test, y_test):
    pred = clf.predict(X_test)
    return json.dumps({
        "r2_score": r2_score(y_test, pred),
        "mse": mean_squared_error(y_test, pred)
    })
    
    
def pipeline():
    wine_data = load_data()
    y = wine_data.quality
    X = wine_data.drop('quality', axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.15, 
                                                        random_state=123, 
                                                        stratify=y)
    clf = dds.keep(path_model, build_model, X_train, y_train)
    dds.keep(path_model_stats, model_stats, clf, X_test, y_test)


dds.eval(pipeline)


DEBUG:dds:locals: ['GridSearchCV', 'In', 'Out', 'RandomForestRegressor', '_', '_2', '_3', '_7', '_8', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_load_data', '_oh', 'build_model', 'data', 'dds', 'exit', 'get_ipython', 'importlib', 'inspect', 'io', 'json', 'load_data', 'logging', 'make_pipeline', 'mean_squared_error', 'model_stats', 'np', 'path_model', 'path_model_stats', 'path_raw', 'pd', 'pipeline', 'preprocessing', 'quit', 'r2_score', 'requests', 'sklearn', 'train_test_split']
DEBUG:dds.introspect:Starting _introspect: <function pipeline at 0x7f21cce80d30>: arg_sig=()
DEBUG:dds.introspect:visit name load_data: skipping (fun/mod)
DEBUG:dds.introspect:visit name train_test_split: skipping (fun/mod)
DEBUG:dds.introspect:visit name dds: skipping (fun/mod)
DEBUG:dds.introspect:is_authorized_path: {'__main__', 'dds'}
DEBUG:dds.i

DEBUG:dds.introspect:keep: None <- <__main__.load_data>: 6d3bc9cdc6555704460b5d584c49f560ae575b6d418b05c2787e9ecf05483561
DEBUG:dds.introspect:fname: ['wine_data', 'drop']
DEBUG:dds.introspect:skipping local var
DEBUG:dds.introspect:fname: ['train_test_split']
DEBUG:dds.introspect:_retrieve_object_rec: ['train_test_split'] <module '__main__'>
DEBUG:dds.introspect:is_authorized_path: {'__main__', 'dds'}
DEBUG:dds.introspect:Actual module <sklearn.model_selection._split> for obj <function train_test_split at 0x7f21d0fdcca0> is not authorized
DEBUG:dds.introspect:skipping none
DEBUG:dds.introspect:fname: ['dds', 'keep']
DEBUG:dds.introspect:_retrieve_object_rec: ['dds', 'keep'] <module '__main__'>
DEBUG:dds.introspect:_retrieve_object_rec: ['keep'] <module 'dds' from '/home/tjhunter/work/dds_py/dds/__init__.py'>
DEBUG:dds.introspect:is_authorized_path: {'__main__', 'dds'}
DEBUG:dds.introspect:Actual module <dds> for obj <function keep at 0x7f21fc1ee160>: authorized
DEBUG:dds.introspect:['

DEBUG:dds.introspect:keep: /wine-quality/my_model_stats.json <- <__main__.model_stats>: 054a1aa4de4a1944069ebf559bbe8c68d66a2965c66ab37df6394705d20d8ba2
DEBUG:dds.introspect:End _introspect: <function pipeline at 0x7f21cce80d30>: FunctionInteractions(fun_body_sig='a49c3838d5420a8ab9262dd79b9054df1abb759e3dba826b20f7a3a181d0703f', fun_return_sig='9afefd7f2939822eb61218f059cd621cc2b76af34b98c7dd20ce56a252919001', fun_context_input_sig=None, outputs=[('/wine-quality/raw', '403f70a01b7aa10148219a87d5d7cd554c8ae84b6e3fafa0476411378e09d60c'), ('/wine-quality/my_model', '7cdcc39020446e16a540e5d29a5375c055a6c0bfebcddb00a8d06a1a682df9a6'), ('/wine-quality/my_model_stats.json', '054a1aa4de4a1944069ebf559bbe8c68d66a2965c66ab37df6394705d20d8ba2')])
DEBUG:dds:Updating path: /wine-quality/raw -> 403f70a01b7aa10148219a87d5d7cd554c8ae84b6e3fafa0476411378e09d60c
DEBUG:dds:Updating path: /wine-quality/my_model -> 7cdcc39020446e16a540e5d29a5375c055a6c0bfebcddb00a8d06a1a682df9a6
DEBUG:dds:Updating path: /

In [10]:
    
    # # 8. Refit on the entire training set
# # No additional code needed if clf.refit == True (default is True)
 
# # 9. Evaluate model pipeline on test data
# pred = clf.predict(X_test)
# print r2_score(y_test, pred)
# print mean_squared_error(y_test, pred)
 
# # 10. Save model for future use
# joblib.dump(clf, 'rf_regressor.pkl')